In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
!nvidia-smi

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "1"

# 1 - Dataset creation

In [ ]:
import os
import glob
import sys
LIB_DIRECTORY = '/root/alok/repos/cv_research/lib/'
sys.path.insert(0, LIB_DIRECTORY)

import pandas as pd
from pycocotools.coco import COCO

In [ ]:
# load relevant file paths for left & right frames and bbox predictions

data_directory_base = '/root/alok/data/images'
stereo_frame_pairs_directory = os.path.join(data_directory_base, 'rectified_stereo_frame_pairs_test_set')
left_image_file_paths = glob.glob('{}/{}/*/input/')

# left_image_file_paths, right_image_file_paths = [], []
# for directory_name in os.listdir(stereo_frame_pairs_directory):
#     directory_path = os.path.join(stereo_frame_pairs_directory, directory_name)
#     left_image_file_path = os.path.join(directory_path, 'input', 'left_frame.jpg')
#     right_image_file_path = os.path.join(directory_path, 'input', 'right_frame.jpg')
#     left_image_file_paths.append(left_image_file_path)
#     right_image_file_paths.append(right_image_file_path)
    
# all_image_path = left_image_file_paths + right_image_file_paths

In [ ]:
dataframe = pd.DataFrame(all_image_path[:], columns=['image path'])
dataframe.head()

<h1> Data Loader </h1>

In [ ]:
from torch.utils.data import Dataset
import numpy as np
import pandas as pd
from PIL import Image
import skimage

In [ ]:
class DataGenerator(Dataset):
    """
    Load a dataset for target estimation
    
    Args:
       - dataframe: DataFrame of image path
    """
    def __init__(self, dataframe):
        self.dataframe = dataframe
    
    def __getitem__(self, index):
        img_path = self.dataframe.iloc[:, 0][index]
        img = skimage.io.imread(img_path)
        
        return img, img_path
    
    def __len__(self):
        return len(self.dataframe)

In [ ]:
data_generator = DataGenerator(dataframe)

# 2 - MaskRCNN forward pass

### 2.1 - Load the model

In [ ]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
import keras

sys.path.insert(0, os.path.join(LIB_DIRECTORY, 'maskrcnn'))
from mrcnn.config import Config
import mrcnn.utils as utils
import mrcnn.model as modellib
import mrcnn.visualize as visualize
from mrcnn.model import log
import mcoco.coco as coco
# import mextra.utils as extra_utils
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

%matplotlib inline
%config IPCompleter.greedy=True
BASE_DIR = '/root/data/models/erko/mask_rcnn_instance_segmentation'
DATA_DIR = '/root/data/erko/'
WEIGHTS_DIR = os.path.join(BASE_DIR, "weights")
MODEL_DIR = os.path.join(BASE_DIR, "logs", "body_part_segmentation_20181112_21H31")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(MODEL_DIR, "mask_rcnn_body_part_segmentation_0117.h5")
print(COCO_MODEL_PATH)
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

In [ ]:
image_size = 1024
rpn_anchor_template = (1, 2, 4, 8, 16) # anchor sizes in pixels
rpn_anchor_scales = tuple(i * (image_size // 16) for i in rpn_anchor_template)

class FishConfig(Config):
    """Configuration for training on the shapes dataset.
    """
    # name your experiments here
    NAME = "full"

    # Train on 1 GPU and 2 images per GPU. Put multiple images on each
    # GPU if the images are small. Batch size is 2 (GPUs * images/GPU).
    GPU_COUNT = 1
    IMAGES_PER_GPU = 2

    # Number of classes (including background)
    NUM_CLASSES = 10  # background + 3 shapes (triangles, circles, and squares)

    # Use smaller images for faster training. 
    IMAGE_MAX_DIM = image_size
    IMAGE_MIN_DIM = image_size
    
    # Use smaller anchors because our image and objects are small
    RPN_ANCHOR_SCALES = rpn_anchor_scales

    # Aim to allow ROI sampling to pick 33% positive ROIs.
    TRAIN_ROIS_PER_IMAGE = 32

    STEPS_PER_EPOCH = 1000

    VALIDATION_STEPS = 300
    
config = FishConfig()
config.display()

In [ ]:
model_path = COCO_MODEL_PATH

In [ ]:
CATEGORIES = [{u'id': 1, u'name': u'F', u'supercategory': u'F'},
    {u'id': 2, u'name': u'Head', u'supercategory': u'Head'},
    {u'id': 3, u'name': u'Caudal Fin', u'supercategory': u'Caudal Fin'},
    {u'id': 4, u'name': u'Dorsal Fin', u'supercategory': u'Dorsal Fin'},
    {u'id': 5, u'name': u'Adipose Fin', u'supercategory': u'Adipose Fin'},
    {u'id': 6, u'name': u'Anal Fin', u'supercategory': u'Anal Fin'},
    {u'id': 7, u'name': u'Pelvic Fin', u'supercategory': u'Pelvic Fin'},
    {u'id': 8, u'name': u'Pectoral Fin', u'supercategory': u'Pectoral Fin'},
    {u'id': 9, u'name': u'Eye', u'supercategory': u'Eye'},
    {u'id': 10, u'name': u'Body', u'supercategory': u'Body'}]

In [ ]:
class InferenceConfig(FishConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

inference_config = InferenceConfig()

# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="inference", 
                          config=inference_config,
                          model_dir=MODEL_DIR)

# Get path to saved weights
# Either set a specific path or find last trained weights
# model_path = os.path.join(ROOT_DIR, ".h5 file name here")
model.find_last()[1]

# Load trained weights (fill in path to trained weights here)
assert model_path != "", "Provide path to trained weights"

# model_path = '/root/data/models/erko/mask_rcnn_instance_segmentation/logs/full_20181002_19H09/mask_rcnn_full_0097.h5'
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)

### 2.3 - Forward pass all images

In [ ]:
import json
import pylab
import matplotlib.pyplot as plt
from tempfile import NamedTemporaryFile
from pycocotools.coco import COCO
from cococreatortools import * 
from datetime import datetime

In [ ]:
from tqdm import tqdm

In [ ]:
INFO = {
    "description": "Fish data",
    "url": "https://github.com/waspinator/pycococreator",
    "version": "0.1.0",
    "year": 2018,
    "contributor": "thossler",
    "date_created": datetime.utcnow().isoformat(' ')
}

LICENSES = [
    {
        "id": 1,
        "name": "Attribution-NonCommercial-ShareAlike License",
        "url": "http://creativecommons.org/licenses/by-nc-sa/2.0/"
    }
]

CATEGORIES = [
    {
        'id': 1,
        'name': 'salmon',
        'supercategory': 'fish',
    }
]

In [ ]:
coco_output = {"info": INFO, "licenses": LICENSES, "categories": CATEGORIES, "images": [], "annotations": []}

In [ ]:
# len(data_generator)
segmentation_id = 1

In [ ]:
for image_id in tqdm(range(len(data_generator))):
    original_image, img_path = data_generator[image_id]
    height,width,_ = original_image.shape
    print(img_path)
    if original_image is not None:
        print(image_id)
        
        results = model.detect([original_image], verbose=1)[0]

        # create the coco stuff
        image_info = create_image_info(image_id, img_path, [width, height])
        coco_output['images'].append(image_info)

        # loop through all the fish detected
        detections_number = len(results['class_ids'])
        for k in range(detections_number):
            # print(segmentation_id)
            category_info = {'id': int(results['class_ids'][k])}
            binary_mask = results['masks'][..., k]
            annotation_info = create_annotation_info(segmentation_id, image_id, category_info, binary_mask,
                                                     [width, height], bounding_box=results['rois'][k, ...], tolerance=2)
            if annotation_info is not None:
                coco_output["annotations"].append(annotation_info)
            segmentation_id += 1


In [ ]:
annotation_file = '/root/alok/data/annotation_file_test_set.json'
with open(annotation_file, 'w') as f:
    json.dump(coco_output, f)

In [ ]:
annotation_file = '/root/alok/data/images/annotation_file_test_set.json'

<h1> Visualize the Results </h1>

In [ ]:
from matplotlib import pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib.collections import PatchCollection

In [ ]:
coco = COCO(annotation_file)

In [ ]:
def get_ax(rows=1, cols=1, size=8):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Change the default size attribute to control the size
    of rendered images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

In [ ]:
def plot_annotations(image_id):    
    image_data = coco.loadImgs([image_id])[0]
    image_file_path = image_data['local_path']
    annotation_ids = coco.getAnnIds(imgIds=[image_id])
    annotations = coco.loadAnns(annotation_ids)

    # load and display instance annotations
    image = skimage.io.imread(image_file_path)
    f, ax = plt.subplots(1, figsize=(20, 20))
    ax.imshow(image)
    ax.axis('off')
    coco.showAnns(annotations)
    
    # display bounding boxes
    for ann in annotations:
        bbox = ann['bbox']
        rectangle = Rectangle((bbox[1], bbox[0]), bbox[3]-bbox[1], bbox[2]-bbox[0], edgecolor='w', facecolor=None, fill=False, linestyle='--', linewidth=2)
        ax.add_patch(rectangle)
#         category_id = ann['category_id']
        category_id = ann['id']
        ax.text(bbox[1], bbox[0] - 10, category_id, fontsize=16, color='w')


    
    


In [ ]:
plot_annotations(527)

In [ ]:
plot_annotations(2222)

<h1> Generate Statistical Analysis of Body Part Segmentations </h1>

In [ ]:
import numpy as np

In [ ]:
def transform_coco_bbox(bbox):
    x1, y1, x2, y2 = bbox[1], bbox[0], bbox[3], bbox[2]
    return x1, y1, x2, y2

def get_centroid_from_coco_bbox(bbox):
    centroid_x = bbox[1] + 0.5 * (bbox[3] - bbox[1])
    centroid_y = bbox[0] + 0.5 * (bbox[2] - bbox[0])
    return (centroid_x, centroid_y)

def determine_if_body_part_falls_inside_detection(centroid, bounding_box):
    return (bounding_box[0] <= centroid[0] <= bounding_box[2]) and (bounding_box[1] <= centroid[1] <= bounding_box[3])

In [ ]:
NUM_CATEGORIES = 10
FULL_FISH_CATEGORY_ID = 1

'''
For each fish detection, determine the body part annotations that correspond to it
'''

image_ids = coco.getImgIds()
images = coco.loadImgs(image_ids)
stereo_frame_pair_ids = [int(image['local_path'].split('/')[-3]) for image in images]
fish_detections = []
unmatched_body_parts = []
for image_id, image, stereo_frame_pair_id in zip(image_ids, images, stereo_frame_pair_ids):
    fish_detections_in_image = []
    annotation_ids = coco.getAnnIds(imgIds=[image_id])
    annotations = coco.loadAnns(annotation_ids)
    full_fish_annotations = [ann for ann in annotations if ann['category_id'] == FULL_FISH_CATEGORY_ID]
    for full_fish_annotation in full_fish_annotations:
        fish_detection = {
            'stereo_frame_pair_id': stereo_frame_pair_id,
            'side': 'left' if 'left' in image['local_path'] else 'right',
            'full_fish_annotation': full_fish_annotation, 
            'body_part_annotations': []
        }
        fish_detection['bounding_box'] = transform_coco_bbox(full_fish_annotation['bbox'])
        fish_detections_in_image.append(fish_detection)
    
    body_part_annotations = [ann for ann in annotations if ann['category_id'] != FULL_FISH_CATEGORY_ID]
    for body_part_annotation in body_part_annotations:
        body_part_centroid = get_centroid_from_coco_bbox(body_part_annotation['bbox'])
        body_part_matched_to_fish_detection = False
        for fish_detection in fish_detections_in_image:
            body_part_inside_detection = \
                determine_if_body_part_falls_inside_detection(body_part_centroid, fish_detection['bounding_box'])
            if body_part_inside_detection:
                fish_detection['body_part_annotations'].append(body_part_annotation)
                body_part_matched_to_fish_detection = True
        if not body_part_matched_to_fish_detection:
            unmatched_body_parts.append(body_part_annotation)
    fish_detections.extend(fish_detections_in_image)
    
            
            
        
    
        

In [ ]:
print('Number of images: {}'.format(len(image_ids)))
print('Number of fish detected per image: {}'.format(len(fish_detections) / float(len(image_ids))))

body_part_annotation_counts = {i: 0 for i in range(1, NUM_CATEGORIES)}
total_body_part_count = 0
number_of_fish_with_all_body_parts_detected = 0
for fish_detection in fish_detections:
    for body_part_annotation in fish_detection['body_part_annotations']:
        body_part_annotation_counts[body_part_annotation['category_id']] += 1
    total_body_part_count += len(fish_detection['body_part_annotations'])
    if total_body_part_count == NUM_CATEGORIES - 1:
        number_of_fish_with_all_body_parts_detected += 1
        
for i in range(1, NUM_CATEGORIES):
    if i != FULL_FISH_CATEGORY_ID:
        body_part_frequency = body_part_annotation_counts[i] / float(len(fish_detections))
        print('Frequency of body part #{} across fish detections: {}'.format(i, body_part_frequency))    
        
print('Average number of body parts detection per fish detection: {}'.format(float(total_body_part_count) / len(fish_detections)))
print('Percentage of fish detections with all body parts detected: {}'.format(float(number_of_fish_with_all_body_parts_detected) / len(fish_detections)))


<h1> Generate frequency breakdown by combination of body parts </h1>

In [ ]:
combinations = []

for fish_detection in fish_detections:
    combination = []
    for body_part_annotation in fish_detection['body_part_annotations']:
        combination.append(body_part_annotation['category_id'])
    combination = sorted(list(set(combination)))
    combinations.append(combination)

combinations = list(set([','.join([str(i) for i in c]) for c in combinations]))[1:]

combination_frequencies = {x: 0 for x in combinations}

for combination in combinations:
    for fish_detection in fish_detections:
        body_part_annotations = [ann['category_id'] for ann in fish_detection['body_part_annotations']]
        if all([c in body_part_annotations for c in [int(i) for i in combination.split(',')]]):
            combination_frequencies[combination] += float(1) / len(fish_detections)
    
frequencies = []
named_combinations = []
unique_combinations = list(combination_frequencies.keys())
for combination in unique_combinations:
    named_combination = []
    for c in [int(i) for i in combination.split(',')]:
        named_combination.append(CATS[c])
    frequencies.append(combination_frequencies[combination])
    named_combinations.append(', '.join(named_combination))

df = pd.DataFrame({'combination': named_combinations, 'frequency': frequencies})
    

In [ ]:
df.sort_values('frequency', ascending=False)

<h1> Determine left-right matches (approach 1) </h1>

In [ ]:
def left_right_matching_1(left_fish_detections, right_fish_detections):
    left_bounding_boxes = [fish['bounding_box'] for fish in left_fish_detections]
    right_bounding_boxes = [fish['bounding_box'] for fish in right_fish_detections]
    
    left_centroids, left_ids = [], []
    for fish in left_fish_detections:
        bbox = fish['bounding_box']
        centroid = [0.5 * (bbox[0] + bbox[2]), 0.5 * (bbox[1] + bbox[3])]
        left_centroids.append(centroid)
        left_ids.append(fish['full_fish_annotation']['id'])
        
    right_centroids, right_ids = [], []
    for fish in right_fish_detections:
        bbox = fish['bounding_box']
        centroid = [0.5 * (bbox[0] + bbox[2]), 0.5 * (bbox[1] + bbox[3])]
        right_centroids.append(centroid)
        right_ids.append(fish['full_fish_annotation']['id'])
        
    for fish in left_fish_detections:
        bbox = fish['bounding_box']
        print(bbox[2] - bbox[0], bbox[3] - bbox[1])
        
    for fish in right_fish_detections:
        bbox = fish['bounding_box']
        print(bbox[2] - bbox[0], bbox[3] - bbox[1])
        
    
    

In [ ]:
def get_bbox_iou(bbox_1, bbox_2):
    bbox_1_length = bbox_1[2] - bbox_1[0]
    bbox_1_height = bbox_1[3] - bbox_1[1]
    bbox_2_length = bbox_2[2] - bbox_2[0]
    bbox_2_height = bbox_2[3] - bbox_2[1]

    a = min(bbox_2[2] - bbox_1[0], bbox_1[2] - bbox_2[0], bbox_1_length, bbox_2_length)
    a = max(a, 0)
    b = min(bbox_2[3] - bbox_1[1], bbox_1[3] - bbox_2[1], bbox_1_height, bbox_2_height)
    b = max(b, 0)

    intersection = a*b
    area_1 = bbox_1_length * bbox_1_height
    area_2 = bbox_2_length * bbox_2_height
    union = area_1 + area_2 - intersection
    iou = float(intersection) / union
    return iou
    

In [ ]:
def left_right_matching_2(left_fish_detections, right_fish_detections):
    for left_fish in left_fish_detections:
        
        left_fish_id = left_fish['full_fish_annotation']['id']
        left_bbox = left_fish['bounding_box']
        
        for right_fish in right_fish_detections:
            right_fish_id = right_fish['full_fish_annotation']['id']
            right_bbox = right_fish['bounding_box']
            if right_bbox[0] < left_bbox[0]:
                translated_right_bbox = (
                    left_bbox[0], 
                    right_bbox[1], 
                    right_bbox[2] - (right_bbox[0] - left_bbox[0]), 
                    right_bbox[3]
                )
                bbox_iou = get_bbox_iou(left_bbox, translated_right_bbox)
                print(left_fish_id, right_fish_id, bbox_iou)

In [ ]:
stereo_frame_pairs = {stereo_frame_pair_id: {'left_fish_detections': [], 'right_fish_detections': []} for stereo_frame_pair_id in list(set(stereo_frame_pair_ids))}
for fish_detection in fish_detections:
    stereo_frame_pair_id, side = fish_detection['stereo_frame_pair_id'], fish_detection['side']
    stereo_frame_pairs[stereo_frame_pair_id]['{}_fish_detections'.format(side)].append(fish_detection)


In [ ]:
for stereo_frame_pair_id in list(stereo_frame_pairs.keys()):
    
    left_fish_detections = stereo_frame_pairs[stereo_frame_pair_id]['left_fish_detections']
    right_fish_detections = stereo_frame_pairs[stereo_frame_pair_id]['right_fish_detections']
    
    if left_fish_detections and right_fish_detections:
        print('Left image id: {}'.format(left_fish_detections[0]['full_fish_annotation']['image_id']))
        print('Right image id: {}'.format(right_fish_detections[0]['full_fish_annotation']['image_id']))
        left_right_matching(left_fish_detections, right_fish_detections)
        print('')
    
    

In [ ]:
results['class_ids']

In [ ]:
image_info

In [ ]:
model.config.BATCH_SIZE

In [ ]:
config.BATCH_SIZE

In [ ]:
with open('/root/data/small_pen_data_collection/body_parts_detection_20181017.json', 'w') as f:
    json.dump(coco_output, f)